In [ ]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
from PIL import Image
import gc

path_git = '/content/ProgettoFVAB'
filename = 'all' #sostituisci con Landmark
#filename = "Landmark"
path_drive = 'C:/Users/CasaLab1/Desktop/Casillo&Natale/DATASET'
dataset_dir = 'dataset_all_2'  #sostituisci con dataset_Landmark
#dataset_dir = 'dataset_Landmark'
LANGUAGES = {
  1:'Italiano',
  2:'Inglese',
  3: 'Tedesco',
  4:'Spagnolo',
  5: 'Olandese',
  6:'Russo',
  7: 'Giapponese'}
LANGUAGES_N = {
    1:0,
    2:1,
    3:2,
    4:3,
    5:4,
    6:5,
    7:6
}
"""
LANGUAGES = {
    4:'Spagnolo',
    7: 'Giapponese'}
LANGUAGES_N = {
    4:0,
    7:1
}"""

In [ ]:
SIZE = (112, 75)
CHANNELS = 3
NBFRAME =200
BS = 4

import glob
import keras
import keras_video
import tensorflow

def load_video(path, group):
  # use sub directories names as classes
  os.chdir(os.path.join(path, group))
  classes = [i.split(os.path.sep)[-1] for i in glob.glob(os.path.join(path, group)+'/*')]
  print(classes)
  classes.sort()
  # some global params
  # pattern to get videos and classes
  glob_pattern=path+'/'+group+'/{classname}/*.avi'
  # for data augmentation
  data_aug = tensorflow.keras.preprocessing.image.ImageDataGenerator(
      zoom_range=.1,
      horizontal_flip=True,
      rotation_range=8,
      width_shift_range=.2,
      height_shift_range=.2)
  # Create video frame generator
  val_shuffle = True
  if group=='test':
    val_shuffle = False
  x = keras_video.VideoFrameGenerator(
      #sequence_time=1,
      classes=classes, 
      nb_frames=NBFRAME,
      glob_pattern=glob_pattern,
      shuffle=val_shuffle,
      #method=keras_video.METHOD_ABS_DIFF, 
      batch_size=BS,
      target_shape=SIZE,
      nb_channel=CHANNELS,
      use_frame_cache=True
      #transformation=data_aug
  )
  return x

In [ ]:
train = load_video(os.path.join(path_drive, dataset_dir), 'train')
validation = load_video(os.path.join(path_drive, dataset_dir), 'validation')
test = load_video(os.path.join(path_drive, dataset_dir), 'test')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D
def build_convnet(shape=(112, 75, 2)):
    momentum = .9
    model = Sequential()
    model.add(Conv2D(64, (3,3), input_shape=shape, padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation="relu"))
    model.add(BatchNormalization(momentum=momentum))
    \
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation="relu"))
    model.add(Conv2D(128, (3,3), padding='same', activation="relu"))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(256, (3,3), padding='same', activation="relu"))
    model.add(Conv2D(256, (3,3), padding='same', activation="relu"))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(512, (3,3), padding='same', activation="relu"))
    model.add(Conv2D(512, (3,3), padding='same', activation="relu"))
    model.add(BatchNormalization(momentum=momentum))
    
    # flatten...
    model.add(GlobalMaxPool2D())
    
    return model

In [ ]:
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout
def action_model(shape=(5, 112, 112, 2), nbout=2):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_convnet(shape[1:])
    
    # then create our final model
    model = Sequential()
    
    # add the convnet with (5, 112, 112, 3) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    model.add(Bidirectional(LSTM(units =512, return_sequences = True, input_shape = (NBFRAME, 112*75*3))))
    model.add(Dropout(0.5))

    # Adding a second LSTM layer and Dropout layer
    model.add(Bidirectional(LSTM(units = 512, return_sequences = True)))
    model.add(Dropout(0.5))

    # Adding a third LSTM layer and Dropout layer
    model.add(Bidirectional(LSTM(units = 512)))
    model.add(Dropout(0.5))
    
    model.add(Dense(nbout, activation='softmax'))
    model.summary()
    return model

In [ ]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 112, 112, 3)
model = action_model(INSHAPE, len(LANGUAGES_N.keys()))
optimizer = tensorflow.keras.optimizers.Adam(lr=0.0001)
model.compile(
    optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
with tensorflow.device('/device:GPU:0'):
    tensorflow.random.set_seed(42)
    EPOCHS=50
    class MyCustomCallback(tensorflow.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            gc.collect()

    es = EarlyStopping(monitor='val_loss',
            min_delta=0,
            patience=80,
            verbose=0, mode='auto')

    history =model.fit_generator(
        train,
        validation_data=validation,
        epochs=EPOCHS,
        steps_per_epoch = len(train),
        validation_steps= len(validation), 
        callbacks = [es]
            #callbacks=[MyCustomCallback()]
    )

In [ ]:
predictions = model.predict_generator(test, steps = len(test))
predictions = predictions.argmax(axis = -1)
print(predictions)

In [ ]:
y_true =[]
x,y = test.next()
for i in range(0,len(test)):
    for j in range(BS):
        y_true.append(y[j])
    x,y = test.next()
y_true = np.array(y_true).argmax(axis = -1)
print(y_true)

In [ ]:
from sklearn.metrics import classification_report

target_names = list(LANGUAGES.values())
print(classification_report(y_true, predictions, target_names=target_names))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

valori=pd.crosstab(predictions, y_true)
fig=plt.figure(figsize=(17,5))
ax1=plt.subplot(121)
ax1.set_title("Confusion Matrix")
sns.heatmap(valori,annot=True,cmap="Blues",cbar=False)
plt.xlabel("attuali") 
plt.ylabel("predetti")

In [ ]:
for j in (list(LANGUAGES_N.keys())):
  i = LANGUAGES_N[j]
  TP= valori.iloc[i,i]
  FP= valori.iloc[i,:].sum()-TP
  FN= valori.iloc[:,i].sum()-TP
  TN= valori.sum().sum() - TP - FP - FN
  print(TP,FP,FN,TN)
  #b=[[TP,FP],[FN,TN]]
  b=[[TP,TN],[FP,FN]]
  fig=plt.figure(figsize=(17,5))
  ax1=plt.subplot(121)
  sns.heatmap(b,annot=True,cmap="Blues",cbar=False,fmt = "g")
  #plt.xlabel("POSITIVI") 
  #plt.ylabel("NEGATIVI")
  tick_marks = [0.5,1.5]
  plt.title(LANGUAGES[j])
  plt.xticks(tick_marks, ["P","N",])
  plt.yticks(tick_marks, ["T","F"])

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc

vis_arr = multilabel_confusion_matrix(y_true, predictions, labels = list(LANGUAGES_N.values()))
print(vis_arr)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in list(LANGUAGES_N.values()):
    fpr[i], tpr[i], _ = roc_curve(y_true == i, predictions == i)
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
# Plot of a ROC curve for a specific class
for i in list(LANGUAGES_N.values()):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()